# NIRS: Neural-based recommender system with a focus on interpretability

In [2]:
import pandas as pd
import gzip

In [3]:
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield json.loads(l)
    
def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('data/Electronics_5.json.gz')

FileNotFoundError: [Errno 2] No such file or directory: 'reviews_Video_Games.json.gz'